In [54]:
import pandas as pd


In [55]:
df1=pd.read_csv("/content/movies.csv")
df2=pd.read_csv("/content/ratings.csv")
merged_df = pd.merge(df1, df2, on="movieId", how="inner")
merged_df.head(5)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [56]:
merged_df.shape

(100836, 6)

In [57]:
merged_df.isnull().any()

movieId      False
title        False
genres       False
userId       False
rating       False
timestamp    False
dtype: bool

In [58]:
merged_df.duplicated().any()

False

In [59]:
from sklearn.preprocessing import LabelEncoder
Encoded=LabelEncoder()
merged_df['encoded_genre']=Encoded.fit_transform(merged_df['genres'])
merged_df.head(5)


,movieId,title,genres,userId,rating,timestamp,encoded_genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,351
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,351
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,351
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,351
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,351


In [60]:
merged_df.duplicated().any()

False

In [61]:
X = merged_df['encoded_genre'].values
y = merged_df['rating'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [62]:
X_train.shape

(80668,)

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
import keras
model=keras.Sequential()
model.add(Embedding(input_dim=len(Encoded.classes_), output_dim=5, input_length=1))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))


In [64]:
model.compile(
              optimizer='SGD',
              loss='mse'
)

In [65]:
model.fit(X_train,y_train,epochs=5,batch_size=32, verbose=0)

In [66]:
loss=model.evaluate(X_test,y_test)
print(loss)

631/631 [==============================] - 1s 2ms/step - loss: 1.0182
1.0181862115859985


In [73]:
user_genre = input("Enter your preferred movie : ")
user_rating = float(input("Enter your preferred movie rating: "))

# Encode the user's genre choice
user_genre_id = Encoded.fit_transform([user_genre])[0]

# Filter movies based on user's genre and rating preferences
recommended_movies = merged_df[(merged_df['encoded_genre'] == user_genre_id) & (merged_df['rating'] >= user_rating)]

# Sort recommended movies by rating in descending order
recommended_movies = recommended_movies.sort_values(by='rating', ascending=False)
if not recommended_movies.empty:
    print("Top Recommended Movies:")
    print(recommended_movies[['title', 'rating']].head(10))
else:
    print("No matching movies found. Please try different preferences.")

Enter your preferred movie : Seven (a.k.a. Se7en) (1995)
Enter your preferred movie rating: 4
Top Recommended Movies:
                                                    title  rating
100181                                    Whiplash (2013)     5.0
98222   Pirates of the Caribbean: Dead Men Tell No Tal...     5.0
100601                                       Black Mirror     5.0
99526   The Adventures of Sherlock Holmes and Doctor W...     5.0
100466            The Godfather Trilogy: 1972-1990 (1992)     5.0
100431                  Death Note: Desu nôto (2006–2007)     5.0
100426                                             Cosmos     5.0
100367                                    Lemonade (2016)     4.5
100495  The Adventures of Sherlock Holmes and Doctor W...     4.5
100467            The Godfather Trilogy: 1972-1990 (1992)     4.5
